In [1]:
tasks_tools_definition = [
    {
        "name": "add_to_shopping_list",
        "description": "Agrega un nuevo artículo a la lista de compras.",
        "parameters": {
            "type": "object",
            "properties": {
                "article": {
                    "type": "string",
                    "description": "El artículo que se agregará a la lista de compras."
                },
                "payday": {
                    "type": "string",
                    "description": "Opcional. Especifica la quincena para agregar el artículo. Formato: 'AAAA-MM-#' donde # es 1 (día 15) o 2 (día 30). Si no se proporciona, se agrega a la siguiente quincena."
                }
            },
            "required": ["article"]
        }
    }
]

In [2]:
from google import genai
from google.genai import types
import dotenv
import os

dotenv.load_dotenv()

client = genai.Client(api_key=os.getenv("GOOGLE_AI_API_KEY"))


from adapters import json_to_part


In [3]:
tools = types.Tool(function_declarations=tasks_tools_definition)
config = types.GenerateContentConfig(tools=[tools])

In [8]:
contents = [
    types.Content(
        role="model",
        parts=[types.Part(text="agrega croquetas a la lista de compras para la quincena del 15 de julio")]
    )
]


In [9]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config
)

In [10]:
tool_call = response.candidates[0].content.parts[0].function_call

In [11]:
tool_call.

SyntaxError: invalid syntax (1126833598.py, line 1)

In [7]:
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": {"article": "frasco de nutella", "payday": "2025-06-2", "status": "success"}},
)

# Append function call and result of the function execution to contents
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

final_response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

print(final_response.text)


Las croquetas se han añadido a la lista de compras para la quincena del 15 de julio.


In [ ]:
contents

[Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='agrega croquetas a la lista de compras para la quincena del 15 de julio')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id=None, args={'payday': '2024-07-1', 'article': 'croquetas'}, name='add_to_shopping_list'), function_response=None, text=None)], role='model'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id=None, name='add_to_shopping_list', response={'result': {'article': 'frasco de nutella', 'payday': '20